In [39]:
import pandas as pd
import json
import requests
import numpy as np

In [40]:
from os import listdir
from os.path import isfile, join
mypath = u'C:\\Users\\aleksey.makarov\\Desktop\\2016-02\\Прогнозирование конверсии\keyphrazeinfo\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [41]:
data = []
for f in onlyfiles:
    with open(mypath+f,'r+') as f:
        data += json.load(f)

In [42]:
for item in data:
    if item['BroadForecasts'] and len(item['BroadForecasts']) > 0:
        for k,v in item['BroadForecasts'][0].iteritems():
            item['BroadForecasts'+k] = v
    if item['ExactForecasts'] and len(item['ExactForecasts']) > 0:
        for k,v in item['ExactForecasts'][0].iteritems():
            item['ExactForecasts'+k] = v
    if item['PageBehaviorFactors'] and len(item['PageBehaviorFactors']) > 0:
        for k,v in item['PageBehaviorFactors'][0].iteritems():
            item['PageBehaviorFactors'+k] = v

In [43]:
df = pd.DataFrame(data)

In [44]:
df2 = pd.read_csv(u'C:\\Users\\aleksey.makarov\\Desktop\\2016-02\\Прогнозирование конверсии\\projects_info.txt',sep='\t',encoding='utf8')

In [45]:
df2.rename(columns={u'YandexDirectID': 'CampaignId'}, inplace=True)

In [46]:
result = pd.merge(df, df2, on=['CampaignId'])

In [115]:
result2 = result

Заполняем NA в поведенческих факторах на среднее

In [116]:
result2['PageBehaviorFactorsAvgVisitDurationSeconds'] = result2['PageBehaviorFactorsAvgVisitDurationSeconds'].fillna(result2['PageBehaviorFactorsAvgVisitDurationSeconds'].mean())
result2['PageBehaviorFactorsBounceRate'] = result2['PageBehaviorFactorsBounceRate'].fillna(result2['PageBehaviorFactorsBounceRate'].mean())
result2['PageBehaviorFactorsPageDepth'] = result2['PageBehaviorFactorsPageDepth'].fillna(result2['PageBehaviorFactorsPageDepth'].mean())

Обрабатываем строки с NA и получаем флаг IsConversion

In [117]:
result2['IsGeoInQuery'] = result2['IsGeoInQuery'].fillna(False)
result2 = result2[pd.notnull(result2['IsAnyWordOnPage'])]
result2 = result2[pd.notnull(result2['IsYandexPageEqualsOur'])]
result2 = result2[pd.notnull(result2['B2Bb2C'])]
result2 = result2[result2['B2Bb2C'] != '0']
result2['DirectCpaVisits'] = pd.to_numeric(result2['DirectCpaVisits'])
result2['IsConversion'] = result2['DirectCpaVisits'] > 0

Убираем данные по ненужным доменам

In [118]:
result3 = result2[~result2[u'Domain'].isin([u'dveri-dvernik.ru',u'dveri-dvernoy.ru',u'ecoplast.moscow','ekonomnye-okna.ru','okna-ecoplast.ru','samson-med.ru'])]

Формируем датафреймы

In [119]:
result3 = result3[['AdjCount','AvgWordLength','B2Bb2C','BroadForecastsMin','BroadForecastsPremiumMax','BroadForecastsShows','ExactForecastsMin','ExactForecastsPremiumMax',
       'ExactForecastsShows','ExactShowsPercent','IsAnyWordOnPage','IsCommerceWords','IsDigits','IsDigitsInEnd','IsDigitsWithWord','IsGeoInQuery',
       'IsMoscow2','IsWordsOnPage','IsYandexPageEqualsOur','Length','NounCount','PageBehaviorFactorsAvgVisitDurationSeconds',
       'PageBehaviorFactorsBounceRate','PageBehaviorFactorsPageDepth','RoundedLength','ServiceType','VerbCount','WordsCount','WordsCountWoStopWords','Brand_y','IsConversion']]

In [121]:
result3['cnv'] = 0
result3.loc[result3["IsConversion"] == True, "cnv"] = 1
result3.to_csv('D:\\prognoz_konversii.csv',encoding='cp1251',sep=';',decimal=',')

In [122]:
result3['AdjCountStr'] = df['AdjCount'].apply(str)
result3['NounCountStr'] = df['NounCount'].apply(str)
result3['VerbCountStr'] = df['VerbCount'].apply(str)
result3['WordsCountStr'] = df['WordsCount'].apply(str)
result3['WordsCountWoStopWordsStr'] = df['WordsCountWoStopWords'].apply(str)

In [123]:
from sklearn import preprocessing
result3 = result3[pd.notnull(result3['BroadForecastsMin'])]
x = result3['BroadForecastsMin'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['BroadForecastsMinScaled'] = x_scaled
result3 = result3[pd.notnull(result3['BroadForecastsPremiumMax'])]
x = result3['BroadForecastsPremiumMax'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['BroadForecastsPremiumMaxScaled'] = x_scaled
result3 = result3[pd.notnull(result3['BroadForecastsShows'])]
x = result3['BroadForecastsShows'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['BroadForecastsShowsScaled'] = x_scaled
result3 = result3[pd.notnull(result3['ExactForecastsMin'])]
x = result3['ExactForecastsMin'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['ExactForecastsMinScaled'] = x_scaled
result3 = result3[pd.notnull(result3['ExactForecastsPremiumMax'])]
x = result3['ExactForecastsPremiumMax'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['ExactForecastsPremiumMaxScaled'] = x_scaled
result3 = result3[pd.notnull(result3['ExactForecastsShows'])]
x = result3['ExactForecastsShows'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['ExactForecastsShowsScaled'] = x_scaled
x = result3['Length'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['LengthScaled'] = x_scaled
x = result3['AvgWordLength'].values.astype(float).reshape((-1, 1))
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result3['AvgWordLengthScaled'] = x_scaled

C:\Users\aleksey.makarov\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [124]:
result3.columns

Index([                                  u'AdjCount',
                                    u'AvgWordLength',
                                           u'B2Bb2C',
                                u'BroadForecastsMin',
                         u'BroadForecastsPremiumMax',
                              u'BroadForecastsShows',
                                u'ExactForecastsMin',
                         u'ExactForecastsPremiumMax',
                              u'ExactForecastsShows',
                                u'ExactShowsPercent',
                                  u'IsAnyWordOnPage',
                                  u'IsCommerceWords',
                                         u'IsDigits',
                                    u'IsDigitsInEnd',
                                 u'IsDigitsWithWord',
                                     u'IsGeoInQuery',
                                        u'IsMoscow2',
                                    u'IsWordsOnPage',
                            

In [59]:
train_df = result3[['AvgWordLengthScaled','B2Bb2C',
       'IsAnyWordOnPage','IsCommerceWords','IsDigits','IsDigitsInEnd','IsDigitsWithWord','IsGeoInQuery',
       'IsMoscow2','IsWordsOnPage','IsYandexPageEqualsOur','LengthScaled','PageBehaviorFactorsAvgVisitDurationSeconds',
       'PageBehaviorFactorsBounceRate','PageBehaviorFactorsPageDepth','RoundedLength','ServiceType','Brand_y','NounCountStr',
                   'VerbCountStr','WordsCountStr','WordsCountWoStopWordsStr','BroadForecastsMinScaled','BroadForecastsPremiumMaxScaled',
                   'BroadForecastsShowsScaled','ExactForecastsMinScaled','ExactForecastsPremiumMaxScaled','ExactForecastsShowsScaled']]
# Датафрейм, у которого не возникает ошибок из-за фич

In [128]:
train_df = result3[['AvgWordLengthScaled','B2Bb2C',
    'IsCommerceWords','IsGeoInQuery','ExactShowsPercent',
       'IsWordsOnPage','IsYandexPageEqualsOur','LengthScaled','PageBehaviorFactorsAvgVisitDurationSeconds',
       'PageBehaviorFactorsBounceRate','PageBehaviorFactorsPageDepth','RoundedLength','ServiceType','Brand_y','NounCountStr',
                   'WordsCountStr','BroadForecastsMinScaled','BroadForecastsPremiumMaxScaled',
                   'BroadForecastsShowsScaled','ExactForecastsMinScaled','ExactForecastsPremiumMaxScaled','ExactForecastsShowsScaled','ExactShowsPercent']]
# Датафрейм, у которого не возникает ошибок из-за фич

In [127]:
list_of_dictionaries = [train_df.iloc[line,:].T.to_dict() for line in range(len(train_df))]

In [129]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()

In [130]:
array = vec.fit_transform(list_of_dictionaries).toarray()

In [131]:
vec.get_feature_names()

['AvgWordLengthScaled',
 u'B2Bb2C=B',
 u'B2Bb2C=C',
 u'Brand_y',
 'BroadForecastsMinScaled',
 'BroadForecastsPremiumMaxScaled',
 'BroadForecastsShowsScaled',
 'ExactForecastsMinScaled',
 'ExactForecastsPremiumMaxScaled',
 'ExactForecastsShowsScaled',
 u'ExactShowsPercent',
 u'IsCommerceWords',
 u'IsGeoInQuery',
 u'IsWordsOnPage',
 u'IsYandexPageEqualsOur',
 'LengthScaled',
 'NounCountStr=0',
 'NounCountStr=1',
 'NounCountStr=2',
 'NounCountStr=3',
 'NounCountStr=4',
 'NounCountStr=5',
 u'PageBehaviorFactorsAvgVisitDurationSeconds',
 u'PageBehaviorFactorsBounceRate',
 u'PageBehaviorFactorsPageDepth',
 u'RoundedLength',
 u'ServiceType=Goods',
 u'ServiceType=GoodsServices',
 u'ServiceType=Services',
 'WordsCountStr=1',
 'WordsCountStr=2',
 'WordsCountStr=3',
 'WordsCountStr=4',
 'WordsCountStr=5',
 'WordsCountStr=6',
 'WordsCountStr=7']

In [132]:
from sklearn import svm
clf = svm.SVC()
clf.fit(array, result3['cnv'])
print clf.score(array, result3['cnv'])

0.984772533813


In [133]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(array, result3['cnv'])
print clf.score(array, result3['cnv'])
result3['RandomForest'] = clf.predict(array)
result3.to_csv('D:\\prognoz_konversii.csv',encoding='cp1251',sep=';',decimal=',')

0.994892651092


In [134]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
clf = GaussianNB()
clf.fit(array, result3['cnv'])
print clf.score(array, result3['cnv'])
result3['GaussianNB'] = clf.predict(array)
result3.to_csv('D:\\prognoz_konversii.csv',encoding='cp1251',sep=';',decimal=',')

0.835240707462


In [135]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(array, result3['cnv'])
print clf.score(array, result3['cnv'])
result3['LDA'] = clf.predict(array)
result3.to_csv('D:\\prognoz_konversii.csv',encoding='cp1251',sep=';',decimal=',')

0.983637567389


In [136]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(array, result3['cnv'])
print model.score(array, result3['cnv'])
result3['ETC'] = model.predict(array)
result3.to_csv('D:\\prognoz_konversii.csv',encoding='cp1251',sep=';',decimal=',')

for x,y in zip(vec.get_feature_names(),model.feature_importances_):
    print x,y

0.997635486617
AvgWordLengthScaled 0.0639302049928
B2Bb2C=B 0.00967837679111
B2Bb2C=C 0.00629801386707
Brand_y 0.0130887798583
BroadForecastsMinScaled 0.067073530861
BroadForecastsPremiumMaxScaled 0.0578638136268
BroadForecastsShowsScaled 0.0620000258473
ExactForecastsMinScaled 0.068525547473
ExactForecastsPremiumMaxScaled 0.056063989436
ExactForecastsShowsScaled 0.0720512452697
ExactShowsPercent 0.0670233989721
IsCommerceWords 0.0193051317903
IsGeoInQuery 0.0150064194947
IsWordsOnPage 0.0207164734265
IsYandexPageEqualsOur 0.013858352794
LengthScaled 0.0554504368013
NounCountStr=0 0.00525080065448
NounCountStr=1 0.0230233595669
NounCountStr=2 0.0174751944977
NounCountStr=3 0.0140864162872
NounCountStr=4 0.00476896698512
NounCountStr=5 0.0
PageBehaviorFactorsAvgVisitDurationSeconds 0.0352182811695
PageBehaviorFactorsBounceRate 0.0393061687471
PageBehaviorFactorsPageDepth 0.04987255442
RoundedLength 0.0465571402317
ServiceType=Goods 0.00773368070608
ServiceType=GoodsServices 0.0060167374

In [137]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    array, result3['cnv'], test_size=0.3, random_state=0)
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
print clf.score(X_test, y_test)
Y_predict = clf.predict(X_test)

0.982976040353


In [145]:
x_df_with_conversions = result3[['AvgWordLengthScaled','B2Bb2C',
    'IsCommerceWords','IsGeoInQuery','ExactShowsPercent',
       'IsWordsOnPage','IsYandexPageEqualsOur','LengthScaled','PageBehaviorFactorsAvgVisitDurationSeconds',
       'PageBehaviorFactorsBounceRate','PageBehaviorFactorsPageDepth','RoundedLength','ServiceType','Brand_y','NounCountStr',
                   'WordsCountStr','BroadForecastsMinScaled','BroadForecastsPremiumMaxScaled',
                   'BroadForecastsShowsScaled','ExactForecastsMinScaled','ExactForecastsPremiumMaxScaled','ExactForecastsShowsScaled','ExactShowsPercent']][result3['cnv'] == 1]
x_df_without_conversions = result3[['AvgWordLengthScaled','B2Bb2C',
    'IsCommerceWords','IsGeoInQuery','ExactShowsPercent',
       'IsWordsOnPage','IsYandexPageEqualsOur','LengthScaled','PageBehaviorFactorsAvgVisitDurationSeconds',
       'PageBehaviorFactorsBounceRate','PageBehaviorFactorsPageDepth','RoundedLength','ServiceType','Brand_y','NounCountStr',
                   'WordsCountStr','BroadForecastsMinScaled','BroadForecastsPremiumMaxScaled',
                   'BroadForecastsShowsScaled','ExactForecastsMinScaled','ExactForecastsPremiumMaxScaled','ExactForecastsShowsScaled','ExactShowsPercent']][result3['cnv'] == 0]
x_of_dictionaries_with_conversions = [x_df_with_conversions.iloc[line,:].T.to_dict() for line in range(len(x_df_with_conversions))]
x_of_dictionaries_without_conversions = [x_df_without_conversions.iloc[line,:].T.to_dict() for line in range(len(x_df_without_conversions))]
vec = DictVectorizer()
x_without_conversions = vec.fit_transform(x_of_dictionaries_without_conversions).toarray()
x_with_conversions = vec.transform(x_of_dictionaries_with_conversions).toarray()
y_with_conversions = result3[result3['cnv'] == 1]['cnv']
y_without_conversions = result3[result3['cnv'] == 0]['cnv']

In [146]:
X_train_with_conversions, X_test_with_conversions, y_train_with_conversions, y_test_with_conversions = cross_validation.train_test_split(
    x_with_conversions, y_with_conversions, test_size=0.3, random_state=0)

In [147]:
X_train_without_conversions, X_test_without_conversions, y_train_without_conversions, y_test_without_conversions = cross_validation.train_test_split(
    x_without_conversions, y_without_conversions, test_size=0.3, random_state=0)

In [149]:
X_train = X_train_with_conversions + X_train_without_conversions

ValueError: operands could not be broadcast together with shapes (112,36) (7288,36) 

In [33]:
len(new_array)

10573

Проверка по Яндекс.XML для запросов, у которых не собраны URL

In [ ]:
from bs4 import BeautifulSoup
import requests
def getXMLData(query,lr):
    query = query.encode('utf-8')
    URL = 'http://seozoo.ru/xmlsearch?user=aleksey.makarov@ingate.ru&key=UH7ForyW2o2QGIruyktzXbiivQ89EsEK197Ioccc&query={0}&lr={1}&l10n=ru&sortby=rlv&filter=moderate&groupby=attr%3D%22%22.mode%3Dflat.groups-on-page%3D100.docs-in-group%3D1'.format(query,lr)
    return requests.get(URL).content
def parseXMLData(data):
    Soup = BeautifulSoup(data)
    results = Soup.find_all('results')[0]
    docs = results.find_all('doc')
    return docs
def getURLfromXML(data):
    Soup = BeautifulSoup(data)
    results = Soup.find_all('results')[0]
    doc = results.find('doc')
    return doc.url.get_text()
df3 = pd.read_csv(u'C:\\Users\\aleksey.makarov\\Desktop\\2016-02\\Прогнозирование конверсии\\for_parse_xml.txt',sep='\t',encoding='utf8')
ld_3 = [df3.iloc[line,:].T.to_dict() for line in range(len(df3))]
for item in ld_3:
    item['query'] = '{0} site:{1}'.format(item['2'].encode('utf-8'),item['3'].encode('utf-8'))
for item in ld_3:
    print item['query']
    try:
        item['url'] = getURLfromXML(getXMLData(item['query'].decode('utf8'),225))
        print item['url']
    except:
        item['url'] = 'error'
        print 'oops'
        continue